In [ ]:
!pip install langchain==0.0.339 openai typing pydantic=="2.5.*" replicate==0.20.* SQLAlchemy==2.0.23 google-generativeai==0.2.* --upgrade


In [ ]:
import os
from langchain.chat_models import ChatOpenAI, ChatGooglePalm
from langchain.prompts import ChatPromptTemplate
from langchain.llms import OpenAI
from langchain import PromptTemplate
from langchain import LLMChain
from langchain.llms import Replicate

from langchain.schema import StrOutputParser
from langchain.cache import SQLiteCache
from langchain.globals import set_llm_cache
from langchain.callbacks import get_openai_callback

from langchain.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)
from langchain.output_parsers import CommaSeparatedListOutputParser


In [ ]:
set_llm_cache(SQLiteCache(database_path=".langchain.db"))


In [150]:
os.environ['OPENAI_API_KEY']="key"

In [142]:
llm_openai = OpenAI(model="gpt-3.5-turbo-instruct")

In [149]:
zero_shot_template = """
You are an helpful e-commerce assistant.
You will judge relevancy of an item to a query and respond as 'relevant' if the item is relevant to the query and 'irrelevant' if the item is not relevant to the query.
query: {query}
item: {item_description}

Let's think step by step and return answer as json.
Answer:
"""


one_shot_template = """
You are an helpful e-commerce assistant.

You will be given example_query and example_item below.

example_query: 30 oz tumbler with handle
example_item: Simple Modern 30 oz Tumbler with Handle and Straw Lid | Insulated Cup Reusable Stainless Steel Water Bottle Travel Cupholder Friendly | Gifts for Women Him Her | Trek Collection | 80s Mix

You will judge relevancy of an item to a query and respond as 'relevant' if the item is relevant to the query and 'irrelevant' if the item is not relevant to the query.

query: {query}
item: {item_description}

Let's think step by step and return answer as json.
Answer: """

prompt = PromptTemplate(template=one_shot_template, input_variables=["query", "item_description"])

In [144]:
chain_gpt4 = LLMChain(llm=llm_openai, prompt=prompt, output_parser=StrOutputParser())


In [145]:
def get_relevancy(query, item):
  with get_openai_callback() as cb:
      result = chain_gpt4.run(query=query,item_description=item)
      return result

In [146]:
query_item_pairs = [{'query': "compression socks women", "item":"Cherokee Workwear Professionals Women Scrubs Pant Mid Rise Straight Leg Drawstring WW160T, XL Tall, Pewter", 'relevancy':'Irrelevant'},
                    {'query': "compression socks women", "item":"3 Pack Copper Compression Socks - Compression Socks Women & Men Circulation - Best for Medical,Running,Athletic", 'relevancy':'Relevant'}
                    ]

In [148]:
get_relevancy(query_item_pairs[1]['query'], query_item_pairs[1]['item'])

'\n{\n  "query": "compression socks women",\n  "item": "3 Pack Copper Compression Socks - Compression Socks Women & Men Circulation - Best for Medical,Running,Athletic",\n  "relevancy": "relevant"\n}'